In [1]:
import getpass
from datetime import datetime
import requests
import pandas as pd
import psycopg2
from psycopg2 import sql, Error, extras

LIST_OF_TICKERS_1 = ['AAPL', 'META', 'NVDA', 'AMD', 'GOOG', 'XOM',
                   'MCD', 'KO', 'PFE', 'PG']

LIST_OF_TICKERS_2 = ['MSFT', 'AMZN', 'TSLA', 'GOOGL', 'JPM', 'JNJ', 'V', 'UNH', 'HD', 'DIS', 'NFLX', 'VZ', 'INTC', 'CMCSA', 'PEP', 'T', 'MRK', 'CSCO']

CREATE_TABLE_SQL = """create table if not exists quotes (
                        id bigint generated by default as identity primary key,
                        quote_date date,
                        open float,
                        high float,
                        low float,
                        close float,
                        volume bigint)"""

INSERT_SQL = """insert into quotes
                  (quote_date, open, high, low, close, volume)
                values %s"""

def main():
    title = "CANDLES DATA LOADER"
    width = 55
    print("=" * width)
    print(title.center(width))
    print("=" * width)
    print("This script allows you to fetch market data via API,")
    print("either in CSV format or by saving it to a database.")
    print("=" * width)
    api_key = getpass.getpass("Enter your API key >>")
    export_method_prompt = (
            "\nEnter the export method for quotes: \n"
            "1 - to a .csv file \n"
            "2 - to a PostgreSQL database \n"
            "3 - quit the programm \n"
        )
    print(export_method_prompt)
    while True:
        export_method = int(input("Enter your command >>"))
        if export_method == 1:
            candles_to_csv(api_key, LIST_OF_TICKERS)
        elif export_method == 2:
            candles_to_db(api_key, LIST_OF_TICKERS)
        elif export_method == 3:
            print("Thanks for using our app!")
            break
        else:
            print("This method doesn't exist")


def candles_to_csv(api_key, tickers):
    result = get_candles_df(api_key, tickers)
    result.to_csv(f'quotes_{datetime.now():%d-%m-%Y}.csv', index=False)
    print(f"Candles had saved in the file quotes_{datetime.now():%d-%m-%Y}.csv")

def candles_to_db(api_key, tickers):
    user, password, host, port, database = get_connection_parameters()
    create_table(user, password, host, port, database, CREATE_TABLE_SQL)
    result = get_candles_df(api_key, tickers)
    save_to_database(user, password, host, port, database, result, INSERT_SQL)

def get_candles_df(api_key, tickers):
    result = pd.DataFrame()
    for ticker in tickers:
      json_data = request_cadles(api_key, ticker)
      df  = pd.DataFrame.from_dict(json_data).T
      df['ticker'] = ticker
      df.rename(columns={
        '1. open':'open',
        '2. high':'high',
        '3. low':'low',
        '4. close':'close',
        '5. volume':'volume'}, inplace=True)
      df.reset_index(inplace=True)
      df.rename(columns={'index': 'date'}, inplace=True)
      result = pd.concat([result, df], ignore_index=True)
    result_filtred = result[result['date'] >= '2021-01-01']
    return result_filtred

def request_cadles(api_key, ticker):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={api_key}"
    response = requests.get(url)
    response.raise_for_status()
    json_data = response.json()
    return json_data['Time Series (Daily)']

def create_table(user, password, host, port, database, sql_query):
    try:
      conn = psycopg2.connect(
            user=user,
            password=password,
            host=host,
            port=port,
            database=database
        )
      cursor = conn.cursor()
      cursor.execute(sql_query)
    except (Exception, Error) as error:
      print(f"Error while working woth PostgreSQL: {error}")
      if conn:
        conn.rollback()
        print("Transaction rolled back due to an error.")
    finally:
      if cursor:
        cursor.close()
      if conn:
        conn.close()

def save_to_database(user, password, host, port, database, df, sql):
    try:
      conn = psycopg2.connect(
            user=user,
            password=password,
            host=host,
            port=port,
            database=database
        )
      cursor = conn.cursor()
      extras.execute_values(cursor, sql,
                            df[['date',
                                'open',
                                'high',
                                'low',
                                'close',
                                'volume']].values)
      conn.commit()
      print("Quote history successfully saved to database")
    except Exception as error:
      print(f"Error while saving data to database: {error}")
      if conn:
        conn.rollback()
        print("Transaction rolled back due to an error")
    finally:
      if cursor:
        cursor.close()
      if conn:
        conn.close()

def get_connection_parameters():
    print("Input the data for connection to database:")
    user = input("username >>")
    password = input("password >>")
    host = input("host >>")
    port = input("port >>")
    database = input("database >>")
    return user, password, host, port, database


if __name__ == "__main__":
    main()



                  CANDLES DATA LOADER                  
This script allows you to fetch market data via API,
either in CSV format or by saving it to a database.

Enter the export method for quotes: 
1 - to a .csv file 
2 - to a PostgreSQL database 
3 - quit the programm 



ValueError: invalid literal for int() with base 10: ''